Author: JC Nacpil

This notebook attaches the latest (2018) population data to the admin 2/3 level osm completeness data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import shapely
import geopandas as gpd
import rasterio
import rasterio.features

import wget

import os
import glob
from zipfile import ZipFile

from datetime import datetime

## Download and open the population data

In [6]:
# Population as of 20181031
pop_url = "https://data.humdata.org/dataset/0b6ce366-8651-43aa-8c36-9f3fdc8d6b4f/resource/8e16e769-2b4d-4e4b-a202-e592d7ff544b/download/mdg_pop_20181031_.xls"

In [11]:
# Download and save to ../download_data
wget.download(pop_url, '../download_data/mdg_pop_20181031_.xls')

'../download_data/mdg_pop_20181031_.xls'

In [3]:
# Open the excel file
pop_df = pd.read_excel('../download_data/mdg_pop_20181031_.xls')
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ADM0_PCODE  1 non-null      object
 1   ADM0_EN     1 non-null      object
 2   p2009       1 non-null      int64 
 3   p2010est    1 non-null      int64 
 4   p2011est    1 non-null      int64 
 5   p2012est    1 non-null      int64 
 6   p2013est    1 non-null      int64 
 7   p2014est    1 non-null      int64 
 8   p2015est    1 non-null      int64 
 9   p2016est    1 non-null      int64 
 10  p2017est    1 non-null      int64 
 11  p2018est    1 non-null      int64 
dtypes: int64(10), object(2)
memory usage: 224.0+ bytes


In [4]:
# Open the excel file specifically for adm2
pop_adm2_df = pd.read_excel('../download_data/mdg_pop_20181031_.xls', sheet_name='mdg_pop_adm2')
pop_adm2_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ADM0_PCODE  119 non-null    object
 1   ADM0_EN     119 non-null    object
 2   ADM1_PCODE  119 non-null    object
 3   ADM1_EN     119 non-null    object
 4   ADM2_PCODE  119 non-null    object
 5   ADM2_EN     119 non-null    object
 6   p2009       119 non-null    int64 
 7   p2010est    119 non-null    int64 
 8   p2011est    119 non-null    int64 
 9   p2012est    119 non-null    int64 
 10  p2013est    119 non-null    int64 
 11  p2014est    119 non-null    int64 
 12  p2015est    119 non-null    int64 
 13  p2016est    119 non-null    int64 
 14  p2017est    119 non-null    int64 
 15  p2018est    119 non-null    int64 
dtypes: int64(10), object(6)
memory usage: 15.0+ KB


In [5]:
# Open the excel file specifically for adm3
pop_adm3_df = pd.read_excel('../download_data/mdg_pop_20181031_.xls', sheet_name='mdg_pop_adm3')
pop_adm3_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1579 entries, 0 to 1578
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ADM0_PCODE  1579 non-null   object
 1   ADM0_EN     1579 non-null   object
 2   ADM1_PCODE  1579 non-null   object
 3   ADM1_EN     1579 non-null   object
 4   ADM2_PCODE  1579 non-null   object
 5   ADM2_EN     1579 non-null   object
 6   ADM2_TYPE   1579 non-null   object
 7   ADM3_PCODE  1579 non-null   object
 8   ADM3_EN     1579 non-null   object
 9   ADM3_TYPE   1579 non-null   object
 10  p2009       1579 non-null   int64 
 11  p2010est    1579 non-null   int64 
 12  p2011est    1579 non-null   int64 
 13  p2012est    1579 non-null   int64 
 14  p2013est    1579 non-null   int64 
 15  p2014est    1579 non-null   int64 
 16  p2015est    1579 non-null   int64 
 17  p2016est    1579 non-null   int64 
 18  p2017est    1579 non-null   int64 
 19  p2018est    1579 non-null   int64 
dtypes: int64

## Load the osm-completeness data

In [7]:
adm2_gdf = gpd.read_file("../data/mapthegap-mdg-adm2-2021-11-23.gpkg",
                            driver='GPKG'
                            )
adm2_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   ADM2_PCODE                            119 non-null    object  
 1   ADM2_EN                               119 non-null    object  
 2   ADM2_TYPE                             119 non-null    object  
 3   pixels_withbuilding_july2021          119 non-null    int64   
 4   pixels_nobuilding_july2021            119 non-null    int64   
 5   percentage_completeness_july2021      119 non-null    float64 
 6   pixels_withbuilding_jan2020           119 non-null    int64   
 7   pixels_nobuilding_jan2020             119 non-null    int64   
 8   percentage_completeness_jan2020       119 non-null    float64 
 9   pixels_withbuilding_november2021      119 non-null    int64   
 10  pixels_nobuilding_november2021        119 non-null    int64   
 11

In [8]:
adm3_gdf = gpd.read_file("../data/mapthegap-mdg-adm3-2021-11-23.gpkg",
                            driver='GPKG'
                            )
adm3_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1579 entries, 0 to 1578
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   ADM3_PCODE                            1579 non-null   object  
 1   ADM3_EN                               1579 non-null   object  
 2   ADM3_TYPE                             1579 non-null   object  
 3   pixels_withbuilding_july2021          1579 non-null   int64   
 4   pixels_nobuilding_july2021            1579 non-null   int64   
 5   percentage_completeness_july2021      1579 non-null   float64 
 6   pixels_withbuilding_jan2020           1579 non-null   int64   
 7   pixels_nobuilding_jan2020             1579 non-null   int64   
 8   percentage_completeness_jan2020       1579 non-null   float64 
 9   ADM2_PCODE                            1579 non-null   object  
 10  pixels_withbuilding_november2021      1579 non-null   int64   
 

## Joining the population data

In [18]:
# Subset pop data (PCODE index, p2018est)
subset_cols = ['ADM2_PCODE','p2018est']

# Left join
adm2_merged_gdf = pd.merge(
    adm2_gdf, 
    pop_adm2_df[subset_cols], 
    how='left', 
    on='ADM2_PCODE'
)

# Rename columns
adm2_merged_gdf = adm2_merged_gdf.rename(columns={'p2018est':'_popsum'})

adm2_merged_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 119 entries, 0 to 118
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   ADM2_PCODE                            119 non-null    object  
 1   ADM2_EN                               119 non-null    object  
 2   ADM2_TYPE                             119 non-null    object  
 3   pixels_withbuilding_july2021          119 non-null    int64   
 4   pixels_nobuilding_july2021            119 non-null    int64   
 5   percentage_completeness_july2021      119 non-null    float64 
 6   pixels_withbuilding_jan2020           119 non-null    int64   
 7   pixels_nobuilding_jan2020             119 non-null    int64   
 8   percentage_completeness_jan2020       119 non-null    float64 
 9   pixels_withbuilding_november2021      119 non-null    int64   
 10  pixels_nobuilding_november2021        119 non-null    int64   
 11

In [19]:
# Save to file
# To gpkg
filename = "../data/mapthegap-mdg-adm2-2021-11-23.gpkg"
adm2_merged_gdf.to_file(filename,driver='GPKG')

filename

'../data/mapthegap-mdg-adm2-2021-11-23.gpkg'

In [20]:
# Subset pop data (PCODE index, p2018est)
subset_cols = ['ADM3_PCODE','p2018est']

# Left join
adm3_merged_gdf = pd.merge(
    adm3_gdf, 
    pop_adm3_df[subset_cols], 
    how='left', 
    on='ADM3_PCODE'
)

# Rename columns
adm3_merged_gdf = adm3_merged_gdf.rename(columns={'p2018est':'_popsum'})

adm3_merged_gdf

,ADM3_PCODE,ADM3_EN,ADM3_TYPE,pixels_withbuilding_july2021,pixels_nobuilding_july2021,percentage_completeness_july2021,pixels_withbuilding_jan2020,pixels_nobuilding_jan2020,percentage_completeness_jan2020,ADM2_PCODE,pixels_withbuilding_november2021,pixels_nobuilding_november2021,percentage_completeness_november2021,geometry,_popsum
0,MG11101001,1er Arrondissement,Commune,3216,990,76.462197,2472,1734,58.773181,MG11101001A,3241,965,77.056586,"POLYGON ((47.50556 -18.89146, 47.50563 -18.891...",310355
1,MG11101002,2e Arrondissement,Commune,1519,3911,27.974217,1439,3991,26.500921,MG11101002A,1516,3914,27.918969,"POLYGON ((47.55842 -18.91178, 47.55857 -18.911...",239056
2,MG11101003,3e Arrondissement,Commune,1456,1742,45.528455,1424,1774,44.527830,MG11101003A,1458,1740,45.590994,"POLYGON ((47.51365 -18.87834, 47.51775 -18.879...",167055
3,MG11101004,4e Arrondissement,Commune,2789,2304,54.761437,1646,3447,32.318869,MG11101004A,2803,2290,55.036324,"POLYGON ((47.50262 -18.91043, 47.50261 -18.910...",318488
4,MG11101005,5e Arrondissement,Commune,3037,6104,33.223936,2153,6988,23.553222,MG11101005A,3043,6098,33.289574,"POLYGON ((47.53500 -18.85464, 47.53518 -18.854...",390355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1574,MG72716270,Amboriala,Commune,419,120,77.736549,0,539,0.000000,MG72716,419,120,77.736549,"POLYGON ((49.43999 -13.77223, 49.44542 -13.773...",12478
1575,MG72716290,Maromokotra Loky,Commune,3,633,0.471698,3,633,0.471698,MG72716,207,429,32.547170,"POLYGON ((49.62261 -12.80568, 49.62282 -12.806...",5972
1576,MG72716310,Antsirabe Nord,Commune,61,2299,2.584746,5,2355,0.211864,MG72716,2036,324,86.271186,"POLYGON ((49.94996 -13.89482, 49.95034 -13.895...",51116
1577,MG72716330,Ampisikinana,Commune,401,220,64.573269,0,621,0.000000,MG72716,401,220,64.573269,"MULTIPOLYGON (((49.83896 -12.85926, 49.83717 -...",6070


In [21]:
# Save to file
# To gpkg
filename = "../data/mapthegap-mdg-adm3-2021-11-23.gpkg"
adm3_merged_gdf.to_file(filename,driver='GPKG')

filename

'../data/mapthegap-mdg-adm3-2021-11-23.gpkg'